# EDB: parameterized design

This example shows how to
1. Set up an HFSS project using SimulationConfiguration class.
2. Create automatically parametrized design.

This image shows the layout created in this example:

<img src="_static\parameterized_design.png" width="600">


## Import dependencies.

In [1]:
import tempfile

In [2]:
import pyaedt

In [3]:
import pyedb
from pyedb.misc.downloads import download_file

## Create an instance of a pyedb.Edb object.

In [4]:
temp_dir = tempfile.TemporaryDirectory(suffix=".ansys")
target_aedb = download_file("edb/ANSYS-HSD_V1.aedb", destination=temp_dir.name)
print("Project is located in ", target_aedb)

# Select EDB version (change it manually if needed, e.g. "2024.1")
edb_version = "2024.1"
print(f"EDB version: {edb_version}")

edb = pyedb.Edb(edbpath=target_aedb, edbversion=edb_version)
print("AEDB file is located in {}".format(target_aedb))

Project is located in  C:\Users\ansys\AppData\Local\Temp\tmp384edbcg.ansys\edb/ANSYS-HSD_V1.aedb
EDB version: 2024.1


PyAEDT INFO: Logger is initialized in EDB.


PyAEDT INFO: legacy v0.21.1


PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyAEDT INFO: Database ANSYS-HSD_V1.aedb Opened in 2024.1


PyAEDT INFO: Cell main Opened


PyAEDT INFO: Builder was initialized.


PyAEDT INFO: EDB initialized.


AEDB file is located in C:\Users\ansys\AppData\Local\Temp\tmp384edbcg.ansys\edb/ANSYS-HSD_V1.aedb


## Prepare the layout for the simulation

The ``new_simulation_configuration()`` method creates an instance of
the ``SimulationConfiguration`` class. This class helps define all preprocessing steps
required to set up the PCB for simulation. After the simulation configuration has been defined,
they are applied to the EDB using the ``Edb.build_simulation()`` method.

In [5]:
simulation_configuration = edb.new_simulation_configuration()
simulation_configuration.signal_nets = [
    "PCIe_Gen4_RX0_P",
    "PCIe_Gen4_RX0_N",
    "PCIe_Gen4_RX1_P",
    "PCIe_Gen4_RX1_N",
]
simulation_configuration.power_nets = ["GND"]
simulation_configuration.components = ["X1", "U1"]
simulation_configuration.do_cutout_subdesign = True
simulation_configuration.start_freq = "OGHz"
simulation_configuration.stop_freq = "20GHz"
simulation_configuration.step_freq = "10MHz"

Now apply the simulation setup to the EDB.

In [6]:
edb.build_simulation_project(simulation_configuration)

PyAEDT INFO: Building simulation project.


PyAEDT INFO: Cutting out using method: 1


PyAEDT INFO: Cutting out using method: 1


PyAEDT INFO: Cutout Multithread started.


PyAEDT INFO: Net clean up Elapsed time: 0m 6sec


...

PyAEDT INFO: Padstack definition with zero plating ratio, defaulting to 20%


PyAEDT INFO: Padstack definition with zero plating ratio, defaulting to 20%


PyAEDT INFO: Padstack definition with zero plating ratio, defaulting to 20%


PyAEDT INFO: EDB file save time: 0.00ms


True

## Parameterize

The layout can automatically be set up to enable parametric studies. For example, the
impact of antipad diameter or trace width on signal integrity performance may be invested
parametrically.

In [7]:
edb.auto_parametrize_design(layers=True, materials=True, via_holes=True, pads=True, antipads=True, traces=True)
edb.save_edb()
edb.close_edb()

PyAEDT INFO: EDB file save time: 0.00ms


PyAEDT INFO: EDB file release time: 0.00ms


True

## Open project in AEDT

All manipulations thus far have been executed using the EDB API, which provides fast,
streamlined processing of layout data in non-graphical mode. The layout and simulation
setup can be visualized by opening it using the 3D Layout editor in AEDT.

Note that there may be some delay while AEDT is being launched.

In [8]:
hfss = pyaedt.Hfss3dLayout(
    projectname=target_aedb,
    specified_version=edb_version,
    non_graphical=False,
    new_desktop_session=True,
)

PyAEDT WARNING: Argument `projectname` is deprecated for method `__init__`; use `project` instead.


PyAEDT WARNING: Argument `specified_version` is deprecated for method `__init__`; use `version` instead.


PyAEDT WARNING: Argument `new_desktop_session` is deprecated for method `__init__`; use `new_desktop` instead.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: StdOut is enabled


...

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyAEDT INFO: AEDT 2024.1.0 Build Date 2023-11-27 22:16:18


PyAEDT INFO: EDB folder C:\Users\ansys\AppData\Local\Temp\tmp384edbcg.ansys\edb/ANSYS-HSD_V1.aedb has been imported to project ANSYS-HSD_V1


PyAEDT INFO: Active Design set to main


PyAEDT INFO: Aedt Objects correctly read


The following cell can be used to ensure that the design is valid for simulation.

In [9]:
validation_info = hfss.validate_full_design()
is_ready_to_simulate = True

PyAEDT INFO: #### Design Validation Checks###


PyAEDT INFO: **** Validation Completed Correctly


PyAEDT INFO: Ports Requested: None


PyAEDT INFO: Ports Defined: 8


PyAEDT INFO: Excitation name: U1.AN26.PCIe_Gen4_RX0_N


...

PyAEDT INFO: Excitation name: U1.AR25.PCIe_Gen4_RX1_P


PyAEDT INFO: Excitation name: X1.A2.PCIe_Gen4_RX0_P


PyAEDT INFO: Excitation name: X1.A3.PCIe_Gen4_RX0_N


PyAEDT INFO: Excitation name: X1.A5.PCIe_Gen4_RX1_P


PyAEDT INFO: Excitation name: X1.A6.PCIe_Gen4_RX1_N


In [10]:
for s in validation_info[0]:
    if "error" in s:
        print(s)
        is_ready_to_simulate = False

if is_ready_to_simulate:
    print("The model is ready for simulation.")
else:
    print("There are errors in the model that must be fixed.")

The model is ready for simulation.


## Release the application from the Python kernel

It is important to release the application from the Python kernel after
execution of the script. The default behavior of the ``release_desktop()`` method closes all open
projects and closes the application.

If you want to continue working on the project in graphical mode
after script execution, call the following method with both arguments set to ``False``.

In [11]:
hfss.release_desktop(close_projects=True, close_desktop=True)
temp_dir.cleanup()  # Remove the temporary folder and files. All data will be removd!

PyAEDT INFO: Desktop has been released and closed.
